In [1]:
cd /home/drew/NBOH/Marzuoli/POPC/Hunter-Sanders

In [10]:
import bioinf as bi
import numpy as np
import random

with open("npt3.pdb") as pdb, open("dry3.pdb", 'w') as w:
    zs = np.array([], dtype=float)
    upper = np.array([], dtype=float)
    lower = np.array([], dtype=float)
    upper_cutoff = None
    lower_cutoff = None
    waters_deleted = []
    z_max = 0
    
    for line in pdb:
        stripped = line.strip()
        pdb_line = bi.parse_pdb_ATOM_line(line)
        if not pdb_line:
            w.write(line)
            continue
        name = pdb_line.get('name', '').strip()
        zstring = pdb_line.get('z', None)
        z = None
        try:
            z = float(zstring)
            z_max = max(z, z_max)
        except:
            w.write(line)
            continue
           
        if name == 'P8': # P head of DPPC
            zs = np.append(zs, z)
        if name == 'OW': #assumes SOL follows all lipids and that only O's belonging to SOL are named 'OW'
            if not upper_cutoff:
                lower, upper = np.split(np.sort(zs), 2) # falsly assumed same number of lipids in top and bottom
                upper_cutoff = np.median(upper) # median is robust against above mistake 
                lower_cutoff = np.median(lower) #TODO: find algorithm to split between modes
            if z < upper_cutoff and z > lower_cutoff:
                pdb.readline() # assumes H1 next
                pdb.readline() # and H2 after that
                waters_deleted.append(pdb_line['resSeq'])
                continue
            w.write(line)
            continue
            
        if name in ["CL", "NA"]: #assumes CL's follow lipids
            if z < upper_cutoff and z > lower_cutoff:
                z = random.choice([random.uniform(0, np.min(lower)), random.uniform(np.max(upper), z_max)])
                pdb_line['z'] = f'{z:.3f}'
            w.write(f'{bi.print_pdb_ATOM_line(pdb_line)}\n')
            continue
        w.write(line)